In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [20]:
df = pd.read_csv('../data/clean_data.csv')

In [21]:
# Handle null values in text column
df['consumer_complaint_narrative'] = df['consumer_complaint_narrative'].fillna('')

vectorizer = TfidfVectorizer(stop_words='english')
x = vectorizer.fit_transform(df['consumer_complaint_narrative'])

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [ ]:
k = 10 ## test diffrent variable amounts
kmeans = KMeans(n_clusters=k, random_state=25)
labels = kmeans.fit_predict(x)

In [ ]:
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

for i in range(k):
    print(f"\nCluster {i} top terms:")
    for j in order_centroids[i, :10]:  # Top 10 terms
        print(terms[j], end=", ")



Cluster 0 top terms:
credit, report, information, xxxx, reporting, identity, accounts, theft, account, inquiries, 
Cluster 1 top terms:
consumer, xxxx, information, 15, reporting, report, consent, code, agency, reports, 
Cluster 2 top terms:
section, usc, 15, 1681, consumer, states, xxxx, reporting, agency, furnish, 
Cluster 3 top terms:
xxxx, xxxxxxxx, account, credit, report, information, reporting, accounts, balance, date, 
Cluster 4 top terms:
debt, xxxx, collection, company, credit, validation, account, proof, alleged, report, 
Cluster 5 top terms:
accounts, credit, report, inaccurate, duty, litigation, fraudulent, accordingly, deny, inaccuracies, 
Cluster 6 top terms:
xxxx, xxxxxxxx, credit, report, items, information, consumer, balance, account, reporting, 
Cluster 7 top terms:
xxxx, account, bank, card, payment, loan, xxxxxxxx, told, credit, called, 
Cluster 8 top terms:
letters, xxxx, complaint, filing, party, involved, im, uploaded, falsely, misleading, 
Cluster 9 top terms: